In [3]:
# Import python's built-in regular expression library
import re
import anthropic
from dotenv import load_dotenv
import os
import json
import re


load_dotenv()

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
API_KEY = os.getenv('ANTHROPIC_API_KEY')
MODEL_NAME = "claude-3-5-sonnet-20241022"

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(messages, tools=[], system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=1000,
        temperature=0.0,
        system=system_prompt,
        messages=messages,
        tools=tools
    )
    return message

### DB CRUD

In [ ]:
# database
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

# crud functions
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

# Engineering tools
def extract_json_from_tags(text):
    pattern = r'<tool_call>\s*(.*?)\s*</tool_call>'
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return json.loads(matches[0])
    else:
        return None
    

FUNCTION_REGISTRY = {
    "add_product": add_product,
    "add_user": add_user,
    "get_product": get_product,
    "get_user": get_user,
}

In [ ]:
AVAILABLE_TOOLS = """
<function_descriptions>
<function>
<name>get_user</name>
<description>Retrieves a user from the database by their unique ID</description>
<parameters>
  <param>
    <name>user_id</name>
    <type>int</type>
    <description>The unique identifier of the user to retrieve</description>
  </param>
</parameters>
<returns>
  <type>dict or None</type>
  <description>Returns a dictionary containing user information (id, name, email) if found, or None if no user with the given ID exists</description>
</returns>
<errors>
  <error>May raise KeyError if the database structure is malformed or missing the "users" key</error>
  <error>May raise TypeError if user_id is not comparable to the stored ID values</error>
</errors>
</function>

<function>
<name>get_product</name>
<description>Retrieves a product from the database by its unique ID</description>
<parameters>
  <param>
    <name>product_id</name>
    <type>int</type>
    <description>The unique identifier of the product to retrieve</description>
  </param>
</parameters>
<returns>
  <type>dict or None</type>
  <description>Returns a dictionary containing product information (id, name, price) if found, or None if no product with the given ID exists</description>
</returns>
<errors>
  <error>May raise KeyError if the database structure is malformed or missing the "products" key</error>
  <error>May raise TypeError if product_id is not comparable to the stored ID values</error>
</errors>
</function>

<function>
<name>add_user</name>
<description>Creates a new user with the provided information and adds it to the database</description>
<parameters>
  <param>
    <name>name</name>
    <type>str</type>
    <description>The name of the new user</description>
  </param>
  <param>
    <name>email</name>
    <type>str</type>
    <description>The email address of the new user</description>
  </param>
</parameters>
<returns>
  <type>dict</type>
  <description>Returns a dictionary containing the newly created user information (id, name, email)</description>
</returns>
<errors>
  <error>May raise KeyError if the database structure is malformed or missing the "users" key</error>
  <error>May raise AttributeError if the database users list doesn't support the append method</error>
</errors>
</function>

<function>
<name>add_product</name>
<description>Creates a new product with the provided information and adds it to the database</description>
<parameters>
  <param>
    <name>name</name>
    <type>str</type>
    <description>The name of the new product</description>
  </param>
  <param>
    <name>price</name>
    <type>float or int</type>
    <description>The price of the new product</description>
  </param>
</parameters>
<returns>
  <type>dict</type>
  <description>Returns a dictionary containing the newly created product information (id, name, price)</description>
</returns>
<errors>
  <error>May raise KeyError if the database structure is malformed or missing the "products" key</error>
  <error>May raise AttributeError if the database products list doesn't support the append method</error>
</errors>
</function>
</function_descriptions>
"""

system_prompt_tools_general_explanation = f"""
You are an db middleware assistant equipped with various tools to help a user interact with the database.
Your goal is to understand the user's request and use the available tools when necessary to provide accurate and helpful responses.
The database stores users and products only.

Here are the tools you have access to:

<available_tools>
{AVAILABLE_TOOLS}
</available_tools>

When responding to the user's query, follow these guidelines:
1. Always consider the available tools and use them when necessary.
2. If no tool is needed or if the available tools cannot help answer the query, respond with "Sorry, I do not have sufficient information at hand to answer this question."
3. If you need to use a tool but there are missing parameters
3.1 get the param name, type and description from the tool description
3.2 respond with "Sorry, we are missing required parameters: " + the param name + " " + the param type + " " + the param description.

Format your response as follows:
1. If you need to use a tool, return the tool call as a JSONschema between <tool_call> and </tool_call> tags and limit to that.
2. If no tool is needed, simply provide your answer.
"""


system_prompt = system_prompt_tools_general_explanation

In [ ]:
# call model with tool use
examples = [
    "Add a user to the database named Deborah with email deborah@example.com.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    messages = [{
            "role": "user",
            "content": "User query: " + example
        },
        ]
    print(messages)

    # Get the function calling response
    function_calling_response = get_completion(messages=messages, system_prompt=system_prompt)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")
    messages.append({
        "role": "assistant",
        "content": function_calling_response
    })

    # Extract the tool call from the function calling response
    tool_call = extract_json_from_tags(function_calling_response)
    print(f"\n*********\nTool to call: {tool_call}\n*********\n")
    if tool_call is None:
        print(f"\n*********\nMissing parameters: {tool_call}\n*********\n")
        continue

    # Call the function
    function_name = tool_call["function"]
    parameters = tool_call["parameters"]
    function_ouput = FUNCTION_REGISTRY[function_name](**parameters)
    print(f"\n*********\nFunction output: {function_ouput}\n*********\n")
    messages.append({
        "role": "user",
        "content": f"Database response: {function_ouput}"
    })

    # Get the final response
    final_response = get_completion(messages=messages, system_prompt=system_prompt)
    print(f"\n*********\nFinal response: {final_response}\n*********\n")


### Simple calculator tool

In [60]:
from anthropic.types.tool_use_block import ToolUseBlock

def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")

tool_registry = {
    "calculator": calculator
}

def execute_tool(tool_call: ToolUseBlock) -> str:
    tool_name = tool_call.name
    tool_input = tool_call.input
    return str(tool_registry[tool_name](**tool_input))

In [ ]:
calculator_tool = {
    "name": "calculator",
    "description": "A calculator that can perform basic arithmetic operations with two operands.",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "description": "The operation word to perform.",
                "enum": ["add", "subtract", "multiply", "divide"]
            },
            "operand1": {
                "type": "number",
                "description": "The first operand"
            },
            "operand2": {
                "type": "number",
                "description": "The second operand"
            }
        },
        "required": ["operation", "operand1", "operand2"]
    }
}

system_prompt = """You have access to tools, but only use them when necessary. If a tool is not required, respond as normal."""

In [ ]:
def prompt_claude(prompt):
    messages = []
    user_message = {
        "role": "user",
        "content": prompt
    }
    messages.append(user_message)
    response = get_completion(messages, tools=[calculator_tool], system_prompt=system_prompt)
    messages.append({
        "role": "assistant",
        "content": response.content
    })
    
    if response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_result = execute_tool(tool_use)
        messages.append({
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": tool_result
                }
            ]
        })
        response = get_completion(messages, tools=[calculator_tool], system_prompt=system_prompt)
    else:
        response
    
    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    return final_response




'145 times 3 equals 435.'

In [76]:
prompt_claude("Hello")

"Hello! I'm here to help you. I have access to a basic calculator that can perform arithmetic operations (addition, subtraction, multiplication, and division) with two numbers. Let me know if you'd like to perform any calculations!"

### Research assistance

In [110]:
import wikipedia
def generate_wikipedia_reading_list(research_topic: str, article_titles: list[str]) -> None:
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t, results=2)
        print("Results:", results)
        try:
            page = wikipedia.page(results[0])
            print("Page:", page)
            title = page.original_title
            print("Title:", title)
            url = page.url
            print("URL:", url)
            wikipedia_articles.append({"title": title, "url": url})
        except wikipedia.exceptions.DisambiguationError as e:
            # If there's ambiguity, take the first option
            page = wikipedia.page(e.options[0])
            print("Page:", page)
            title = page.original_title
            print("Title:", title)
            url = page.url
            print("URL:", url)
            wikipedia_articles.append({"title": title, "url": url})
        except wikipedia.exceptions.PageError:
            # Skip if page doesn't exist
            continue
    print("Wikipedia articles:", wikipedia_articles)
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles: list[dict], topic: str) -> None:
    with open("output/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")

tool_registry = {
    "generate_wikipedia_reading_list": generate_wikipedia_reading_list
}

def execute_tool(tool_call: ToolUseBlock) -> str:
    tool_name = tool_call.name
    tool_input = tool_call.input
    return str(tool_registry[tool_name](**tool_input))

In [ ]:
system_prompt = """
You are a research assistant.
You task is to generate relevant article names related to a topic. 
Then you will use the tools provided to you to search if the articles you invented actually exists in wikipedia.

If the articles you invented actually exists in wikipedia, it will be added to the research reading file.

The article names must be in english and related to the topic, think on possible article titles from wikipedia.
"""

generate_wikipedia_reading_list_tool_description = {
    "name": "generate_wikipedia_reading_list",
    "description": "Based on a topic it will generate a list of articles from wikipedia.",
    "input_schema": {
        "type": "object",
        "properties": {
            "research_topic": {
                "type": "string",
                "description": "The topic of the research"
            },
            "article_titles": {
                "type": "array",
                "description": "The list of article titles related to the research topic to search in wikipedia",
                "items": {
                    "type": "string"
                }
            }
        },
        "required": ["research_topic", "article_titles"]
    }
}

def get_research_help(topic, number_of_articles):
    messages = []
    user_message = {
        "role": "user",
        "content": f"Research topic: {topic}, Number of articles to generate: {number_of_articles}"
    }
    messages.append(user_message)
    response = get_completion(messages, tools=[generate_wikipedia_reading_list_tool_description], system_prompt=system_prompt)
    print("Claude tool use:", next(block for block in response.content if block.type == "tool_use"))
    messages.append({
        "role": "assistant",
        "content": response.content
    })
    if response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        execute_tool(tool_use)
        messages.append({
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": "The articles were generated successfully"
                }
            ]
        })
        response = get_completion(messages, system_prompt=system_prompt)
    else:
        response
    
    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    return response

In [120]:
get_research_help("The Simpsons", 10)

Claude tool use: Message(id='msg_01VbdqLcbbZ8zKgX3UffqWdn', content=[TextBlock(citations=None, text="I'll help you generate a list of relevant Wikipedia articles related to The Simpsons. I'll create 10 potential article titles that are likely to exist on Wikipedia and then verify them using the generate_wikipedia_reading_list tool.\n\nLet me suggest some key article titles that should cover various aspects of The Simpsons:", type='text'), ToolUseBlock(id='toolu_016S9cTtXgggmtyJnSp3vbss', input={'research_topic': 'The Simpsons', 'article_titles': ['The Simpsons', 'Matt Groening', 'List of The Simpsons episodes', 'Springfield (The Simpsons)', 'Homer Simpson', 'Bart Simpson', 'Lisa Simpson', 'Marge Simpson', 'The Simpsons Movie', 'Simpsons Comics']}, name='generate_wikipedia_reading_list', type='tool_use')], model='claude-3-5-sonnet-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0,

Message(id='msg_012HjCjFFKP5WWR74VXAhLiz', content=[TextBlock(citations=None, text='Great! All of these articles exist on Wikipedia and have been added to the reading list. These articles cover:\n\n1. The main series article ("The Simpsons")\n2. The show\'s creator (Matt Groening)\n3. Episode guide ("List of The Simpsons episodes")\n4. The show\'s setting ("Springfield (The Simpsons)")\n5. Main family members (Homer, Bart, Lisa, and Marge Simpson)\n6. The feature film ("The Simpsons Movie")\n7. The comic book series ("Simpsons Comics")\n\nThese articles should provide a comprehensive overview of The Simpsons franchise, its characters, and its cultural impact. Would you like me to generate more article suggestions or focus on a specific aspect of The Simpsons?', type='text')], model='claude-3-5-sonnet-20241022', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=344, output_toke

### Structured outputs

##### Sentiment scores

In [133]:
tools = [
    {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given text.",
        "input_schema": {
            "type": "object",
            "properties": {
                "positive_score": {"type": "number", "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
                "negative_score": {"type": "number", "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
                "neutral_score": {"type": "number", "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"]
        }
    }
]

tool_choice={"type": "tool", "name": "print_sentiment_scores"}


In [136]:
from anthropic import Anthropic
from dotenv import load_dotenv
import json

load_dotenv()
client = Anthropic()

def analyze_sentiment(tweet: str) -> None:
    query = f"""
    <text>
    {tweet}
    </text>
    """

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=4096,
        tool_choice=tool_choice,
        tools=tools,
        messages=[{"role": "user", "content": query}]
    )
    json_sentiment = None
    for content in response.content:
        if content.type == "tool_use" and content.name == "print_sentiment_scores":
            json_sentiment = content.input
            break

    if json_sentiment:
        print("Sentiment Analysis (JSON):")
        print(json.dumps(json_sentiment, indent=2))
    else:
        print("No sentiment analysis found in the response.")

In [137]:
analyze_sentiment("I'm a HUGE hater of pickles.  I actually despise pickles.  They are garbage.")

Sentiment Analysis (JSON):
{
  "positive_score": 0.1,
  "negative_score": 0.8,
  "neutral_score": 0.1
}


In [138]:
analyze_sentiment("Honestly I have no opinion on taking baths")

Sentiment Analysis (JSON):
{
  "positive_score": 0.0,
  "negative_score": 0.0,
  "neutral_score": 1.0
}


##### Wikipedia summary

In [139]:
import wikipedia

#tool definition
tools = [
    {
        "name": "print_article_classification",
        "description": "Prints the classification results.",
        "input_schema": {
            "type": "object",
            "properties": {
                "subject": {
                    "type": "string",
                    "description": "The overall subject of the article",
                },
                "summary": {
                    "type": "string",
                    "description": "A paragaph summary of the article"
                },
                "keywords": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "description": "List of keywords and topics in the article"
                    }
                },
                "categories": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "The category name."},
                            "score": {"type": "number", "description": "The classification score for the category, ranging from 0.0 to 1.0."}
                        },
                        "required": ["name", "score"]
                    }
                }
            },
            "required": ["subject","summary", "keywords", "categories"]
        }
    }
]

#The function that generates the json for a given article subject
def generate_json_for_article(subject):
    page = wikipedia.page(subject, auto_suggest=True)
    query = f"""
    <document>
    {page.content}
    </document>

    Use the print_article_classification tool. Example categories are Politics, Sports, Technology, Entertainment, Business.
    """

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=4096,
        tools=tools,
        messages=[{"role": "user", "content": query}]
    )

    json_classification = None
    for content in response.content:
        if content.type == "tool_use" and content.name == "print_article_classification":
            json_classification = content.input
            break

    if json_classification:
        print("Text Classification (JSON):")
        print(json.dumps(json_classification, indent=2))
    else:
        print("No text classification found in the response.")

In [141]:
generate_json_for_article("Spiderman")


Text Classification (JSON):
{
  "subject": "Spider-Man",
  "summary": "The article provides a detailed history and overview of the Marvel Comics superhero Spider-Man, including his creation, development, powers, supporting cast, enemies, romantic interests, and legacy. It covers his origins, early adventures, rise to fame, alternate versions, and appearances across various media.",
  "keywords": [
    "Spider-Man",
    "Marvel Comics",
    "Stan Lee",
    "Steve Ditko",
    "superheroes",
    "comic books",
    "secret identity",
    "Peter Parker"
  ],
  "categories": [
    {
      "name": "Entertainment",
      "score": 0.9
    },
    {
      "name": "Comics",
      "score": 0.8
    },
    {
      "name": "Popular Culture",
      "score": 0.7
    }
  ]
}


##### Translator

In [ ]:
tools = [
    {
        "name": "translate_text",
        "description": "Translate text to another language.",
        "input_schema": {
            "type": "object",
            "properties": {
                "english": {"type": "string", "description": "Translate the input text to english."},
                "spanish": {"type": "string", "description": "Translate the input text to spanish."},
                "french": {"type": "string", "description": "Translate the input text to french."},
                "german": {"type": "string", "description": "Translate the input text to german."},
                "japanese": {"type": "string", "description": "Translate the input text to japanese."},
                "arabic": {"type": "string", "description": "Translate the input text to arabic."},
            },
            "required": ["english", "spanish", "french", "german", "japanese", "arabic"]
        }
    }
]

tool_choice = {"type": "tool", "name": "translate_text"}

def translate(text: str) -> None:
    query = f"""
    <input_text>
    {text}
    </input_text>
    """
    
    messages = []
    messages.append({
        "role": "user",
        "content": query
    })
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=200,
        tools=tools,
        tool_choice=tool_choice,
        messages=messages
    )
    json_translation = None
    print("Response:", response.content)
    for content in response.content:
        if content.type == "tool_use" and content.name == "translate_text":
            json_translation = content.input
            break
    if json_translation:
        print("Translation (JSON):")
        print(json.dumps(json_translation, ensure_ascii=False, indent=2))
    else:
        print("No translation found in the response.")

In [159]:
translate("La AI nos va a reemplazar")

Response: [ToolUseBlock(id='toolu_01WWeWT3WpV3EDcJsfaDPhh7', input={'english': 'AI is going to replace us', 'spanish': 'La AI nos va a reemplazar', 'french': "L'IA va nous remplacer", 'german': 'KI wird uns ersetzen', 'japanese': 'AIは私たちを置き換えるでしょう', 'arabic': 'الذكاء الاصطناعي سوف يحل محلنا'}, name='translate_text', type='tool_use')]
Translation (JSON):
{'english': 'AI is going to replace us', 'spanish': 'La AI nos va a reemplazar', 'french': "L'IA va nous remplacer", 'german': 'KI wird uns ersetzen', 'japanese': 'AIは私たちを置き換えるでしょう', 'arabic': 'الذكاء الاصطناعي سوف يحل محلنا'}
{
  "english": "AI is going to replace us",
  "spanish": "La AI nos va a reemplazar",
  "french": "L'IA va nous remplacer",
  "german": "KI wird uns ersetzen",
  "japanese": "AIは私たちを置き換えるでしょう",
  "arabic": "الذكاء الاصطناعي سوف يحل محلنا"
}


##### Tool use with multiple tools

In [200]:
class FakeDatabase:
    def __init__(self):
        self.customers = [
            {"id": "1213210", "name": "John Doe", "email": "john@gmail.com", "phone": "123-456-7890", "username": "johndoe"},
            {"id": "2837622", "name": "Priya Patel", "email": "priya@candy.com", "phone": "987-654-3210", "username": "priya123"},
            {"id": "3924156", "name": "Liam Nguyen", "email": "lnguyen@yahoo.com", "phone": "555-123-4567", "username": "liamn"},
            {"id": "4782901", "name": "Aaliyah Davis", "email": "aaliyahd@hotmail.com", "phone": "111-222-3333", "username": "adavis"},
            {"id": "5190753", "name": "Hiroshi Nakamura", "email": "hiroshi@gmail.com", "phone": "444-555-6666", "username": "hiroshin"},
            {"id": "6824095", "name": "Fatima Ahmed", "email": "fatimaa@outlook.com", "phone": "777-888-9999", "username": "fatimaahmed"},
            {"id": "7135680", "name": "Alejandro Rodriguez", "email": "arodriguez@protonmail.com", "phone": "222-333-4444", "username": "alexr"},
            {"id": "8259147", "name": "Megan Anderson", "email": "megana@gmail.com", "phone": "666-777-8888", "username": "manderson"},
            {"id": "9603481", "name": "Kwame Osei", "email": "kwameo@yahoo.com", "phone": "999-000-1111", "username": "kwameo"},
            {"id": "1057426", "name": "Mei Lin", "email": "meilin@gmail.com", "phone": "333-444-5555", "username": "mlin"}
        ]

        self.orders = [
            {"id": "24601", "customer_id": "1213210", "product": "Wireless Headphones", "quantity": 1, "price": 79.99, "status": "Shipped"},
            {"id": "13579", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 2, "price": 19.99, "status": "Processing"},
            {"id": "97531", "customer_id": "2837622", "product": "Bluetooth Speaker", "quantity": 1, "price": "49.99", "status": "Shipped"}, 
            {"id": "86420", "customer_id": "3924156", "product": "Fitness Tracker", "quantity": 1, "price": 129.99, "status": "Delivered"},
            {"id": "54321", "customer_id": "4782901", "product": "Laptop Sleeve", "quantity": 3, "price": 24.99, "status": "Shipped"},
            {"id": "19283", "customer_id": "5190753", "product": "Wireless Mouse", "quantity": 1, "price": 34.99, "status": "Processing"},
            {"id": "74651", "customer_id": "6824095", "product": "Gaming Keyboard", "quantity": 1, "price": 89.99, "status": "Delivered"},
            {"id": "30298", "customer_id": "7135680", "product": "Portable Charger", "quantity": 2, "price": 29.99, "status": "Shipped"},
            {"id": "47652", "customer_id": "8259147", "product": "Smartwatch", "quantity": 1, "price": 199.99, "status": "Processing"},
            {"id": "61984", "customer_id": "9603481", "product": "Noise-Cancelling Headphones", "quantity": 1, "price": 149.99, "status": "Shipped"},
            {"id": "58243", "customer_id": "1057426", "product": "Wireless Earbuds", "quantity": 2, "price": 99.99, "status": "Delivered"},
            {"id": "90357", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 1, "price": 19.99, "status": "Shipped"},
            {"id": "28164", "customer_id": "2837622", "product": "Wireless Headphones", "quantity": 2, "price": 79.99, "status": "Processing"}
        ]

    def get_user(self, key, value):
        if key in {"email", "phone", "username"}:
            for customer in self.customers:
                if customer[key] == value:
                    return customer
            return f"Couldn't find a user with {key} of {value}"
        else:
            raise ValueError(f"Invalid key: {key}")
        
        return None

    def get_order_by_id(self, order_id):
        for order in self.orders:
            if order["id"] == order_id:
                return order
        return None
    
    def get_customer_orders(self, customer_id):
        return [order for order in self.orders if order["customer_id"] == customer_id]

    def cancel_order(self, order_id):
        order = self.get_order_by_id(order_id)
        if order:
            if order["status"] == "Processing":
                order["status"] = "Cancelled"
                return "Cancelled the order"
            else:
                return "Order has already shipped.  Can't cancel it."
        return "Can't find that order!"

def execute_tool(tool_call: ToolUseBlock, tool_registry: dict) -> str:
    tool_name = tool_call.name
    tool_input = tool_call.input
    return tool_registry[tool_name](**tool_input)

In [ ]:
db = FakeDatabase()

tool_registry = {
    "get_user": db.get_user,
    "get_order_by_id": db.get_order_by_id,
    "get_customer_orders": db.get_customer_orders,
    "cancel_order": db.cancel_order
}

In [199]:
tools = [
    {
        "name": "get_user",
        "description": "Looks up a user by email, phone, or username.",
        "input_schema": {
            "type": "object",
            "properties": {
                "key": {
                    "type": "string", 
                    "enum": ["email", "phone", "username"],
                    "description": "The attribute to search for a user by (email, phone, or username)."
                    },
                "value": {
                    "type": "string", 
                    "description": "The value to match for the specified attribute."
                    }
            },  
            "required": ["key", "value"]
        }
    },
    {
        "name": "get_order_by_id",
        "description": "Looks up an order by its ID.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {"type": "string", "description": "The ID of the order to look up."}
            },
        }
    },
    {
        "name": "get_customer_orders",
        "description": "Looks up all orders belonging to a customer by their ID.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {"type": "string", "description": "The customer_id of the customer to look up."}
            },
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order by its ID. Only orders that are in the 'Processing' state can be cancelled.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {"type": "string", "description": "The order_id of the order to cancel."}
            },
        }
    }
]